In [1]:
from keras.datasets import imdb
import numpy as np
from sklearn.decomposition import NMF
from keras.models import Sequential, Model, Input
from keras.layers import Dense, Activation, Bidirectional, Add, Concatenate, Embedding, Lambda, concatenate, Dropout, TimeDistributed
from keras.layers import LSTM, Multiply, Lambda,TimeDistributed,GRU,Multiply,Activation
from keras.optimizers import Adam
from keras.layers.core import Masking
from keras import backend as K
from keras import losses
import pandas as pd
import tensorflow as tf
from dateutil.parser import parse
import datetime
import copy
from keras.layers import LeakyReLU
import random


import tensorflow as tf
config = tf.ConfigProto()
config.gpu_options.allow_growth = True  # dynamically grow the memory used on the GPU
config.log_device_placement = True  # to log device placement (on which device the operation ran)
sess = tf.Session(config=config)
K.tensorflow_backend.set_session(sess)

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:34: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
embed_size = 10
hidden_size = 20
sequential_code_channel_size=3
logical_variables = [2,3,5,6,7]
logical_operator = {
    0:'not',
    1:'',
}
none_var = 0
num_vocab= len(logical_operator)+len(logical_variables)+1
batch_size=50
episode_len = 10
num_epoch = 1000000
verbose=False

In [3]:
def print_log(*args):
    if verbose:
        print(args)

In [4]:
max_operations = 2

In [5]:
max_length = 4
exp = []

def generate_expression(max_length,indent='-'):
    cnf = []
    for i in range(max_length):
        var_1 = random.choice(logical_variables)
        var_2 = random.choice(logical_variables)
        var_1_not = random.choice(list(logical_operator.keys()))
        var_2_not = random.choice(list(logical_operator.keys()))
        cnf.append([var_1_not,var_1,var_2_not,var_2])
    return cnf

In [6]:
exp = generate_expression(5, '-')
print('exp:',exp)

exp: [[0, 6, 0, 7], [0, 5, 0, 6], [1, 2, 0, 3], [0, 3, 0, 7], [1, 6, 1, 7]]


In [7]:
def generate_readable_expression(cnf,idx=0):
    exp_readable = []
    for df in cnf:
        disjunctions = []
        disjunctions.append('(')
        disjunctions.append(logical_operator[df[0]])
        disjunctions.append(str(df[1]))
        disjunctions.append('or')
        disjunctions.append(logical_operator[df[2]])
        disjunctions.append(str(df[3]))
        disjunctions.append(')')
        exp_readable.append(' '.join(disjunctions))
    return ' and '.join(exp_readable).strip(),_

def generate_readable_expression_(exp,idx=0):
    org_idx=idx
    exp_readable = []
    if idx>=len(exp):
        return exp_readable,idx
    if exp[idx] in logical_operator.keys():
        if exp[idx]==1:
            exp_readable.append(logical_operator[exp[idx]][0])
            exp_1,idx_end = generate_readable_expression(exp,idx+1)
            exp_readable.extend(exp_1)
            idx = idx_end
        else:
            exp_readable.append('(')
            exp_1,idx_end = generate_readable_expression(exp,idx+1)
            exp_readable.extend(exp_1)
            exp_readable.append(logical_operator[exp[idx]][0])
            exp_2,idx_end = generate_readable_expression(exp,idx_end)
            exp_readable.extend(exp_2)
            exp_readable.append(')')
            idx = idx_end
    else:
        if exp[idx]!=0: #-check?
            exp_readable.append(exp[idx])
        idx+=1
    if org_idx==0 and idx!=len(exp) and exp[idx]!=0: #-check?
        exp_readable=[]
    return exp_readable,idx

In [8]:
generate_readable_expression([[1, 5, 1, 6], [0, 6, 1, 5], [1, 5, 1, 5], [1, 5, 0, 5], [0, 6, 0, 6]])

('(  5 or  6 ) and ( not 6 or  5 ) and (  5 or  5 ) and (  5 or not 5 ) and ( not 6 or not 6 )',
 '')

In [9]:
verbose = True
exp = generate_expression(5, '-')
print('exp:',exp)
exp_readable,_ = generate_readable_expression(exp)
print('exp_readable:',exp_readable)

exp: [[0, 2, 1, 3], [1, 5, 0, 2], [0, 3, 0, 2], [0, 7, 0, 2], [1, 6, 1, 2]]
exp_readable: ( not 2 or  3 ) and (  5 or not 2 ) and ( not 3 or not 2 ) and ( not 7 or not 2 ) and (  6 or  2 )


In [10]:
def simulator_generator_example(max_ip_seq_length,max_comp_seq_length):
    batch = []
    while True:
        exp = generate_expression(max_ip_seq_length)
        batch.append(exp)
        if len(batch) % batch_size ==0 and len(batch)!=0:
            batch = np.asarray(batch)
            batch = [batch[:,:,0:1],batch[:,:,1:2],batch[:,:,2:3],batch[:,:,3:]]
            yield batch
            batch = []
            
def simulator_generator_example_(max_ip_seq_length,max_comp_seq_length):
    batch = []
    while True:
        batch.append(generate_expression(max_ip_seq_length))
        if len(batch) % batch_size ==0 and len(batch)!=0:
            batch = np.expand_dims(batch,axis=-1)
            yield batch
            batch = []

In [11]:
gen = simulator_generator_example(4,10)

In [12]:
verbose=False
a = gen.__next__()
[np.asarray(z[0]).flatten() for z in a]

[array([0, 0, 0, 0]),
 array([5, 2, 6, 2]),
 array([0, 0, 0, 0]),
 array([6, 5, 3, 6])]

In [13]:
possible_val_combinations = [[]]
for i in range(len(logical_variables)):
    agumented_possible_val_combinations = []
    for comb in possible_val_combinations:
        agumented_possible_val_combinations.append(comb.copy()+[True])
        agumented_possible_val_combinations.append(comb.copy()+[False])
    possible_val_combinations=agumented_possible_val_combinations

def score_function(expression1,expression2):
    exp_1_readable,_ = generate_readable_expression(expression1)
    exp_2_readable,_ = generate_readable_expression(expression2)

    
    print_log('exp_1_readable:',exp_1_readable)
    print_log('exp_2_readable:',exp_2_readable)
    
    score = 0
    #exp_equal = False
    
    for comb in possible_val_combinations:
        val_exp_1 = exp_1_readable
        val_exp_2 = exp_2_readable
        for i,var in enumerate(logical_variables):
            val_exp_1 = val_exp_1.replace(str(var),str(comb[i]))
            val_exp_2 = val_exp_2.replace(str(var),str(comb[i]))
            
        print_log('val_exp_1:',val_exp_1)
        print_log('val_exp_2:',val_exp_2)

        sol1=eval(val_exp_1.strip())
        sol2=eval(val_exp_2.strip())
        
        
        exp_equal = sol1==sol2
        #if not exp_equal:
        #    break
        if sol1==sol2:
            #score=1
            score+=1
        else:
            score-=1
            #score=-1
            #break
    
    print_log('sol1:',sol1)
    print_log('sol2:',sol2)
    # non-linear scaling
    #score = score*np.abs(score)
    
    return score

In [14]:
score_function([[1, 3, 1, 2], [0, 3, 1, 3], [1, 2, 0, 2], [1, 3, 0, 3], [1, 3, 0, 2]],[[1, 3, 1, 2], [0, 3, 1, 3], [1, 2, 0, 2], [1, 3, 0, 3], [1, 3, 0, 2]])

32

In [15]:
verbose=True
score_function([[1, 3, 1, 2], [0, 3, 1, 3], [1, 2, 0, 2], [1, 3, 0, 3], [1, 3, 0, 2]],[[1, 3, 1, 2], [0, 3, 1, 3], [1, 2, 0, 2], [1, 3, 0, 3], [0, 3, 0, 2]])

('exp_1_readable:', '(  3 or  2 ) and ( not 3 or  3 ) and (  2 or not 2 ) and (  3 or not 3 ) and (  3 or not 2 )')
('exp_2_readable:', '(  3 or  2 ) and ( not 3 or  3 ) and (  2 or not 2 ) and (  3 or not 3 ) and ( not 3 or not 2 )')
('val_exp_1:', '(  True or  True ) and ( not True or  True ) and (  True or not True ) and (  True or not True ) and (  True or not True )')
('val_exp_2:', '(  True or  True ) and ( not True or  True ) and (  True or not True ) and (  True or not True ) and ( not True or not True )')
('val_exp_1:', '(  True or  True ) and ( not True or  True ) and (  True or not True ) and (  True or not True ) and (  True or not True )')
('val_exp_2:', '(  True or  True ) and ( not True or  True ) and (  True or not True ) and (  True or not True ) and ( not True or not True )')
('val_exp_1:', '(  True or  True ) and ( not True or  True ) and (  True or not True ) and (  True or not True ) and (  True or not True )')
('val_exp_2:', '(  True or  True ) and ( not True or  

0

In [16]:
def get_encoder_agent():
    x_i_not_1 = Input(shape=(None,1))
    x_i_var_1 = Input(shape=(None,1))
    x_i_not_2 = Input(shape=(None,1))
    x_i_var_2 = Input(shape=(None,1))
    
    x_o = Input(shape=(None,1))
    
    var_emebed = Embedding(num_vocab, embed_size)

    y_var_1 = TimeDistributed(var_emebed)(x_i_var_1)
    y_var_1 = Lambda(lambda z:K.squeeze(z,axis=-2))(y_var_1)
    
    y_var_2 = TimeDistributed(var_emebed)(x_i_var_2)
    y_var_2 = Lambda(lambda z:K.squeeze(z,axis=-2))(y_var_2)
    
    
    print('x_i_not_1:',K.int_shape(x_i_not_1))
    print('y_var_1:',K.int_shape(y_var_1))
    print('x_i_not_1:',K.int_shape(x_i_not_2))
    print('y_var_2:',K.int_shape(y_var_2))
    
    y = Concatenate(axis=-1)([x_i_not_1,y_var_1,x_i_not_2,y_var_2])
    
    #y = LSTM(hidden_size,return_sequences=True)(y)
    y,h,c = LSTM(hidden_size,return_state=True)(y)
    
    print('x_o:',K.int_shape(x_o))
    y_o=TimeDistributed(Embedding(sequential_code_channel_size, embed_size))(x_o)
    y_o = Lambda(lambda z:K.squeeze(z,axis=-2))(y_o)
    print('y_o:',K.int_shape(y_o))
    
    y,_,_ = LSTM(hidden_size,return_sequences=True, return_state=True)(y_o,initial_state=[h,c])
    #y = LSTM(hidden_size,return_sequences=True)(y)
    
    #y = TimeDistributed(Dense(sequential_code_channel_size,activation='softmax'))(y)
    y = TimeDistributed(Dense(sequential_code_channel_size,activation=None))(y)
    
    
    y = TimeDistributed(Activation('softmax'))(y)
    
    model = Model(inputs=[x_i_not_1,x_i_var_1,x_i_not_2,x_i_var_2,x_o],outputs=y)
    
    return model

In [17]:
def get_decoder_agent():
    x_i = Input(shape=(None,sequential_code_channel_size))
    x_o_not_1 = Input(shape=(None,1))
    x_o_var_1 = Input(shape=(None,1))
    x_o_not_2 = Input(shape=(None,1))
    x_o_var_2 = Input(shape=(None,1))
    
    y = TimeDistributed(Dense(embed_size))(x_i) #Embedding layer
    
    print('emebd:',K.int_shape(y))
    
    #y = Lambda(lambda z:K.squeeze(z,axis=-2))(y)
    #y = LSTM(hidden_size,return_sequences=True)(y)
    y,h,c = LSTM(hidden_size,return_state=True)(y)
    
    
    ##--------------seq-inp-starts-----------------
    var_emebed = Embedding(num_vocab, embed_size)
    
    y_var_1 = TimeDistributed(var_emebed)(x_o_var_1)
    y_var_1 = Lambda(lambda z:K.squeeze(z,axis=-2))(y_var_1)
    
    y_var_2 = TimeDistributed(var_emebed)(x_o_var_2)
    y_var_2 = Lambda(lambda z:K.squeeze(z,axis=-2))(y_var_2)
    
    print('x_i_not_1:',K.int_shape(x_o_not_1))
    print('y_var_1:',K.int_shape(y_var_1))
    print('x_i_not_1:',K.int_shape(x_o_not_2))
    print('y_var_2:',K.int_shape(y_var_2))
    
    y_o = Concatenate(axis=-1)([x_o_not_1,y_var_1,x_o_not_2,y_var_2])
    
    print('y_o:',K.int_shape(y_o))
    ##--------------seq-inp-end-----------------

    
    y,_,_ = LSTM(hidden_size,return_sequences=True, return_state=True)(y_o,initial_state=[h,c])
    #y = LSTM(hidden_size,return_sequences=True)(y)
    
    y_not_1 = TimeDistributed(Dense(1,activation=None))(y)
    y_not_2 = TimeDistributed(Dense(1,activation=None))(y)
    
    y_var_1 = TimeDistributed(Dense(num_vocab,activation=None))(y)
    y_var_2 = TimeDistributed(Dense(num_vocab,activation=None))(y)
    
    y_not_1_activated = TimeDistributed(Activation('sigmoid'))(y_not_1)
    y_not_2_activated = TimeDistributed(Activation('sigmoid'))(y_not_2)
    
    y_var_1_activated = TimeDistributed(Activation('softmax'))(y_var_1)
    y_var_2_activated = TimeDistributed(Activation('softmax'))(y_var_2)
    
    model = Model(inputs=[x_i,
                          x_o_not_1,
                          x_o_var_1,
                          x_o_not_2,
                          x_o_var_2],
                  outputs=[y_not_1,
                           y_var_1,
                           y_not_2,
                           y_var_2])
    
    return model

In [18]:
encoder_agent = get_encoder_agent()
encoder_agent.summary()

x_i_not_1: (None, None, 1)
y_var_1: (None, None, 10)
x_i_not_1: (None, None, 1)
y_var_2: (None, None, 10)
x_o: (None, None, 1)
y_o: (None, None, 10)
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_2 (InputLayer)             (None, None, 1)       0                                            
____________________________________________________________________________________________________
input_4 (InputLayer)             (None, None, 1)       0                                            
____________________________________________________________________________________________________
time_distributed_1 (TimeDistribu (None, None, 1, 10)   80          input_2[0][0]                    
____________________________________________________________________________________________________
time_distributed_2 (TimeDistribu (None, Non

In [19]:
decoder_agent = get_decoder_agent()
decoder_agent.summary()

emebd: (None, None, 10)
x_i_not_1: (None, None, 1)
y_var_1: (None, None, 10)
x_i_not_1: (None, None, 1)
y_var_2: (None, None, 10)
y_o: (None, None, 22)
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_8 (InputLayer)             (None, None, 1)       0                                            
____________________________________________________________________________________________________
input_10 (InputLayer)            (None, None, 1)       0                                            
____________________________________________________________________________________________________
time_distributed_7 (TimeDistribu (None, None, 1, 10)   80          input_8[0][0]                    
____________________________________________________________________________________________________
time_distributed_8 (TimeDistribu (None, 

In [20]:
decoder_agent.compile(optimizer='adam', loss='categorical_crossentropy')
encoder_agent.compile(optimizer='adam', loss='categorical_crossentropy')

In [21]:
from keras.callbacks import TensorBoard,ReduceLROnPlateau

tb_callback = TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)

In [22]:
def sparse_to_dense(sparse,reward,num_dims):
    dense = np.zeros((sparse.shape[0],sparse.shape[1],num_dims))
    for i in range(sparse.shape[0]):
        for j in range(sparse.shape[1]):
            #if sparse[i,j,0] == 0: #-check?
            #    break
            dense[i,j,sparse[i,j,0]]=reward[i]
    return dense
    

In [23]:
sparse_to_dense(np.asarray([[[1],[0]]]),[-1,1],2)

array([[[ 0., -1.],
        [-1.,  0.]]])

In [24]:
max_ip_seq_length = 4
max_seq_communication_length = 10
max_op_seq_length = 4

In [25]:
!rm -rf poorboard_full_diff_CNF.txt

In [26]:
f= open("poorboard_full_diff_CNF.txt","w+")
def print_board(*args):
    f.write(','.join([str(z) for z in args])+'\n')
    f.flush()

In [27]:
encoder_inp_not_1 = Input(shape=(None,1))
encoder_inp_var_1 = Input(shape=(None,1))
encoder_inp_not_2 = Input(shape=(None,1))
encoder_inp_var_2 = Input(shape=(None,1))
encoder_seq_gen_inp = Input(shape=(None,1))
sample_inp = Input(shape=(None,sequential_code_channel_size))
decoder_seq_gen_inp_not_1 = Input(shape=(None,1))
decoder_seq_gen_inp_var_1 = Input(shape=(None,1))
decoder_seq_gen_inp_not_2 = Input(shape=(None,1))
decoder_seq_gen_inp_var_2 = Input(shape=(None,1))


y_encoder_pred = encoder_agent([encoder_inp_not_1,
                                encoder_inp_var_1,
                                encoder_inp_not_2,
                                encoder_inp_var_2,
                                encoder_seq_gen_inp])

#y_reparam = Multiply()([y_encoder_pred,sample_inp])
y_reparam = y_encoder_pred

#y_reparam = y_encoder_pred

print('y_reparam:',K.int_shape(y_reparam))

#print('decoder_seq_gen_inp:',K.int_shape(decoder_seq_gen_inp),decoder_seq_gen_inp)
y = decoder_agent([y_reparam,
                   decoder_seq_gen_inp_not_1,
                  decoder_seq_gen_inp_var_1,
                  decoder_seq_gen_inp_not_2,
                  decoder_seq_gen_inp_var_2])
print('y:',y)
end_end_model = Model([encoder_inp_not_1,
                       encoder_inp_var_1,
                       encoder_inp_not_2,
                       encoder_inp_var_2,
                       encoder_seq_gen_inp,
                       sample_inp,
                       decoder_seq_gen_inp_not_1,
                       decoder_seq_gen_inp_var_1,
                      decoder_seq_gen_inp_not_2,
                      decoder_seq_gen_inp_var_2],y)

end_end_model.summary()

y_reparam: (None, None, 3)
y: [<tf.Tensor 'model_2/time_distributed_9/Reshape_1:0' shape=(?, ?, 1) dtype=float32>, <tf.Tensor 'model_2/time_distributed_11/Reshape_1:0' shape=(?, ?, 8) dtype=float32>, <tf.Tensor 'model_2/time_distributed_10/Reshape_1:0' shape=(?, ?, 1) dtype=float32>, <tf.Tensor 'model_2/time_distributed_12/Reshape_1:0' shape=(?, ?, 8) dtype=float32>]
____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_11 (InputLayer)            (None, None, 1)       0                                            
____________________________________________________________________________________________________
input_12 (InputLayer)            (None, None, 1)       0                                            
____________________________________________________________________________________________________
input_13 (InputLayer)   

In [28]:
end_end_model.compile(optimizer='adam', loss=['binary_crossentropy',
                                              'categorical_crossentropy',
                                              'binary_crossentropy',
                                              'categorical_crossentropy'
                                             ])

In [29]:
def predict_each_step_encode(agent,inp,max_seq_len):
    seq_gen_inp_initial=np.zeros((batch_size,1,1))
    seq_gen_inp = seq_gen_inp_initial.copy()
    for p in range(0,max_seq_len):   
        #print('encoder_seq_gen_inp:',encoder_seq_gen_inp.shape)
        inp_agumented = inp.copy()
        inp_agumented.append(seq_gen_inp)
        
        output = agent.predict(inp_agumented)
        
        
        
        output_disctretised = np.argmax(output,axis=-1)
        output_disctretised=np.expand_dims(output_disctretised,axis=-1)
        seq_gen_inp = np.concatenate((seq_gen_inp_initial, output_disctretised), axis=-2)
    seq_gen_inp=seq_gen_inp[:,:-1,:]
    return seq_gen_inp,output,output_disctretised


def predict_each_step_decode(agent,inp,max_seq_len):
    seq_gen_inp_initial=[np.zeros((batch_size,1,1)) for z in range(4)] #-check?
    seq_gen_inp = seq_gen_inp_initial.copy()
    for p in range(0,max_seq_len):   
        #print('encoder_seq_gen_inp:',encoder_seq_gen_inp.shape)
        inp_agumented = inp.copy()
        inp_agumented.extend(seq_gen_inp)
        
        #print('inp:',[z.shape for z in inp_agumented])
        
        output = agent.predict(inp_agumented)
        
        #print('output:',[z.shape for z in output])
        
        
        output_disctretised = [np.expand_dims(np.argmax(z,axis=-1),axis=-1) for z in output]
        
        #print('output_disctretised:',[z.shape for z in output_disctretised])
        
        seq_gen_inp = np.concatenate((seq_gen_inp_initial, output_disctretised), axis=-2)
    seq_gen_inp=[z[:,:-1,:] for z in seq_gen_inp]
    #print('#_# seq_gen_inp:',[z.shape for z in seq_gen_inp])
    return seq_gen_inp,output,output_disctretised


In [ ]:
verbose = 0
batch_size=50
sim_gen = simulator_generator_example(max_ip_seq_length,max_seq_communication_length)
num_epoch_steps=1

for epoch_num in range(0,num_epoch,num_epoch_steps):
    encoder_inp = sim_gen.__next__()
    
    encoder_seq_gen_inp,encoder_output,encoder_output_disctretised=predict_each_step_encode(encoder_agent,encoder_inp,max_seq_communication_length)
    decoder_seq_gen_inp,decoder_output,decoder_output_disctretised=predict_each_step_decode(decoder_agent,[encoder_output],max_op_seq_length)

    
    #print('encoder_input:',encoder_inp[0].flatten())
    #print('encoder_seq_gen_inp:',encoder_seq_gen_inp[0].flatten())
    #print('encoder_output:',encoder_output_disctretised[0].flatten())
    #print('decoder_output:',decoder_output_disctretised[0].flatten())
    
    #print('encoder_inp:',np.asarray(encoder_inp).shape)
    #print('encoder_inp:',[z.shape for z in encoder_inp])
    
    #print('=>decoder_output_disctretised:',np.asarray(decoder_output_disctretised).shape)
    
    batch_inp = np.squeeze(encoder_inp,axis=-1)
    batch_inp = np.einsum('fbl->blf',batch_inp)
    batch_op = np.squeeze(decoder_output_disctretised,axis=-1)
    batch_op = np.einsum('fbl->blf',batch_op)
    
    #print('batch_inp:',batch_inp.shape)
    #print('batch_op:',batch_op.shape)
    
    rewards = []
    for idx,(ip_exp,op_exp) in enumerate(zip(batch_inp,batch_op)):
    #for idx in range(batch_size):
        try:
            #print('ip_exp:',ip_exp.shape)
            #print('op_exp:',ip_exp.shape)
            
            reward = score_function(ip_exp,op_exp)
        except Exception as e:
            print('T_T ',e,':',ip_exp,':',op_exp)
            reward = -1000
        rewards.append(reward)
    
    #print('input:',ip_exp)
    #print('output:',op_exp)
    
    #print('rewards:',np.asarray(rewards).shape)
    
    print_board(np.sum(rewards),np.mean(rewards))
    
    def sparse_to_dense(sparse,reward,num_dims):
        dense = np.zeros((sparse.shape[0],sparse.shape[1],num_dims))
        for i in range(sparse.shape[0]):
            for j in range(sparse.shape[1]):
                #if sparse[i,j,0] == 0: #-check?
                #    break
                dense[i,j,sparse[i,j,0]]=reward[i]
        return dense
    
    #encoder_reward = sparse_to_dense(encoder_output_disctretised,rewards,sequential_code_channel_size)
    
    decoder_reward=[]
    decoder_reward.append( np.asarray([u*v for u,v in zip(decoder_output_disctretised[0],rewards)]))
    decoder_reward.append(sparse_to_dense(np.expand_dims(decoder_output_disctretised[1],axis=-1),rewards,num_vocab))
    decoder_reward.append( np.asarray([u*v for u,v in zip(decoder_output_disctretised[2],rewards)]))
    decoder_reward.append(sparse_to_dense(np.expand_dims(decoder_output_disctretised[3],axis=-1),rewards,num_vocab))
    
    #print('decoder_reward:',[z.shape for z in decoder_reward])
    sampling_input = sparse_to_dense(encoder_output_disctretised,np.ones(batch_size),sequential_code_channel_size)

    
#     import collections
#     print('decoder_reward:',collections.Counter(decoder_reward.flatten()))

#     import pdb; pdb.set_trace()
    #print('encoder_inp:',encoder_inp.shape)
    #print('encoder_seq_gen_inp:',encoder_seq_gen_inp.shape)
    #print('sampling_input:',sampling_input.shape)
    #print('decoder_seq_gen_inp:',decoder_seq_gen_inp.shape)
    
    #print('encoder_inp:',[z.shape for z in encoder_inp])
    #print('encoder_seq_gen_inp:',encoder_seq_gen_inp.shape)
    #print('sampling_input:',sampling_input.shape)
    #print('decoder_seq_gen_inp:',[z.shape for z in decoder_seq_gen_inp])
    
    
       
    end_end_model.fit([encoder_inp[0],
                       encoder_inp[1],
                       encoder_inp[2],
                       encoder_inp[3],
                       encoder_seq_gen_inp,
                       sampling_input,
                       decoder_seq_gen_inp[0],
                      decoder_seq_gen_inp[1],
                      decoder_seq_gen_inp[2],
                      decoder_seq_gen_inp[3]],
                      
                      decoder_reward,
                      initial_epoch = num_epoch,
                      epochs=num_epoch+num_epoch_steps,
                      #callbacks=[tb_callback],
                      verbose=verbose)


In [31]:
print(generate_readable_expression(batch_inp[0]))
print(generate_readable_expression(batch_inp[1]))

('( not 2 or not 2 ) and ( not 3 or  3 ) and (  2 or not 2 ) and (  3 or  3 )', array([[[ 0., -1.],
        [-1.,  0.]]]))
('(  3 or not 2 ) and ( not 3 or not 3 ) and (  3 or not 3 ) and (  3 or not 2 )', array([[[ 0., -1.],
        [-1.,  0.]]]))


In [32]:
np.argmax(encoder_output[0],axis=-1)

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [33]:
encoder_output[1]

array([[0.307485  , 0.28382766, 0.40868732],
       [0.31341413, 0.28543827, 0.40114754],
       [0.31783542, 0.28850314, 0.39366144],
       [0.32115772, 0.29198703, 0.38685524],
       [0.32366627, 0.2953838 , 0.38094994],
       [0.32558048, 0.29844677, 0.37597272],
       [0.3270603 , 0.30108273, 0.37185696],
       [0.32821897, 0.30328336, 0.36849773],
       [0.32913572, 0.30508313, 0.36578116],
       [0.32986656, 0.30653438, 0.3635991 ]], dtype=float32)

In [34]:
sampling_input[0]

array([[0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.],
       [0., 0., 1.]])

In [ ]:
print('decoder_output:',decoder_output_disctretised)

In [106]:
!head -n 375 /usr/local/lib/python3.5/dist-packages/tensorflow/python/framework/tensor_util.py

# Copyright 2015 The TensorFlow Authors. All Rights Reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.
# ==============================================================================

"""Utilities to create TensorProtos."""
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import six

from tensorflow.core.framework import tensor_pb2
from tensorflow.core.framework import tensor_shape_pb2
from tensorflow.python.fr

In [ ]:
verbose = 0
sim_gen = simulator_generator_example(max_ip_seq_length,max_seq_communication_length)
num_epoch_steps=1


for epoch_num in range(0,num_epoch,num_epoch_steps):
    batch = sim_gen.__next__()
    
    
    encoder_output = encoder_agent.predict(batch)
    encoder_output_disctretised = np.argmax(encoder_output,axis=-1)
    encoder_output_disctretised=np.expand_dims(encoder_output_disctretised,axis=-1)
    
    
    dummy_decoder_input = np.zeros((encoder_output_disctretised.shape[0],max_op_seq_length,1))
    decoder_output=decoder_agent.predict([encoder_output_disctretised,dummy_decoder_input])

    decoder_output_discrete = np.argmax(decoder_output,axis=-1)
    
    print('encoder_output:',encoder_output_disctretised[0].flatten())
    print('decoder_output:',decoder_output_discrete[0])
    
    rewards = []
    for idx,(ip_exp,op_exp) in enumerate(zip(batch[0],decoder_output_discrete)):
        try:
            ip_exp=np.squeeze(ip_exp,axis=-1)
            #print('ip_exp:',ip_exp)
            reward = score_function(ip_exp,op_exp)
            if reward:
                reward = 1
            else:
                reward = -1
        except Exception as e:
            print(e)
            reward = -1
        rewards.append(reward)
    
    #print('input:',ip_exp)
    #print('output:',op_exp)
    
    
    print(np.sum(rewards),np.mean(rewards))
    encoder_reward = sparse_to_dense(encoder_output_disctretised,rewards,sequential_code_channel_size)
    decoder_reward = sparse_to_dense(np.expand_dims(decoder_output_discrete,axis=-1),rewards,num_vocab)
    
    
    #import pdb; pdb.set_trace()
    
    encoder_agent.fit(batch,encoder_reward,
                      initial_epoch = num_epoch,
                      epochs=num_epoch+num_epoch_steps,
                      #callbacks=[tb_callback],
                      verbose=verbose)
    decoder_agent.fit([encoder_output_disctretised,dummy_decoder_input],decoder_reward,
                      initial_epoch = num_epoch,
                      epochs=num_epoch+num_epoch_steps,
                      #callbacks=[tb_callback],
                      verbose=verbose)


In [ ]:
#     encoder_seq_gen_inp_initial=np.zeros((batch_size,1,1))
#     encoder_seq_gen_inp = encoder_seq_gen_inp_initial.copy()
#     for p in range(0,max_seq_communication_length):   
#         #print('encoder_seq_gen_inp:',encoder_seq_gen_inp.shape)
#         encoder_output = encoder_agent.predict([encoder_inp,encoder_seq_gen_inp])
#         encoder_output_disctretised = np.argmax(encoder_output,axis=-1)
#         encoder_output_disctretised=np.expand_dims(encoder_output_disctretised,axis=-1)
#         encoder_seq_gen_inp = np.concatenate((encoder_seq_gen_inp_initial, encoder_output_disctretised), axis=-2)
#     encoder_seq_gen_inp=encoder_seq_gen_inp[:,:-1,:]
    
    encoder_seq_gen_inp,encoder_output_disctretised=predict_each_step(encoder_agent,encoder_inp,max_seq_communication_length)
    decoder_seq_gen_inp,decoder_output_disctretised=predict_each_step(decoder_agent,encoder_output_disctretised,max_op_seq_length)
    
    
#     decoder_seq_gen_inp_initial=np.zeros((batch_size,1,1))
#     decoder_seq_gen_inp = decoder_seq_gen_inp_initial.copy()
#     for p in range(0,max_op_seq_length):    
#         #print('decoder_seq_gen_inp:',decoder_seq_gen_inp.shape)
#         decoder_output=decoder_agent.predict([encoder_output_disctretised,decoder_seq_gen_inp])
#         decoder_output_disctretised = np.argmax(decoder_output,axis=-1)
#         decoder_output_disctretised=np.expand_dims(decoder_output_disctretised,axis=-1)
#         decoder_seq_gen_inp = np.concatenate((decoder_seq_gen_inp_initial, decoder_output_disctretised), axis=-2)
#     decoder_seq_gen_inp=decoder_seq_gen_inp[:,:-1,:]
    
    #print('encoder_input:',encoder_inp[0].flatten())
    #print('encoder_seq_gen_inp:',encoder_seq_gen_inp[0].flatten())
    #print('encoder_output:',encoder_output_disctretised[0].flatten())
    #print('decoder_output:',decoder_output_disctretised[0].flatten())

In [96]:
a = np.asarray([[3],[3],[3]])
b = [0,0.1,0.05]

In [101]:
np.asarray([z*u for z,u in zip(a,b)]).shape

(3, 1)